In [2]:
# Test Wikipedia extract function from your prompt builder (copy-paste into a notebook cell)

from pathlib import Path
import importlib.util
import time


# --- Load the prompt builder module directly from file (works even if folders are not Python packages)
MODULE_PATH = Path("../inference/task_a/two_words/prompt_builder_two_words.py").resolve()

spec = importlib.util.spec_from_file_location("prompt_builder_two_words", str(MODULE_PATH))
if spec is None or spec.loader is None:
    raise RuntimeError(f"Could not load module from: {MODULE_PATH}")

pb = importlib.util.module_from_spec(spec)
spec.loader.exec_module(pb)  # type: ignore

print("Loaded:", MODULE_PATH)


# --- Quick sanity checks
print("safe_word('Bar!') ->", pb.safe_word("Bar!"))
print("safe_word('table') ->", pb.safe_word("table"))


# --- Test get_wikipedia_extract_cached
words = ["bar", "table", "SQL", "NoSQL", "banana", "cat"]

for w in words:
    t0 = time.perf_counter()
    first = pb.get_wikipedia_extract_cached(w)
    t1 = time.perf_counter()

    second = pb.get_wikipedia_extract_cached(w)  # should be cached in-process
    t2 = time.perf_counter()

    print("\nWORD:", repr(w))
    print("First call seconds:", round(t1 - t0, 4))
    print("Second call seconds:", round(t2 - t1, 4))
    print("Extract preview:", (first[:200] + "...") if first else "<empty>")
    if first != second:
        print("Warning: cached result differs from first result")


# --- Optional: test the facts block builder that uses the same cached extracts
print("\n--- FACTS BLOCK EXAMPLE ---")
print(pb.format_facts_block("bar", "table"))


In [6]:
import requests

url = "https://en.wikipedia.org/api/rest_v1/page/summary/cat"
try:
    headers = {
    "User-Agent": "MWAHAHA/1.0 (contact: dardemtum@gmail.com) humor-generation"
    }
    r = requests.get(url, headers=headers, timeout=8)
    print("status:", r.status_code)
    print("content-type:", r.headers.get("content-type"))
    print("text preview:", r.text[:1000])
except Exception as e:
    print("request failed:", repr(e))
